# Problem 1

We want to implement P1-FEM in 1D. For benchmarking, we use a simple Poisson problem in 1D:
$$
    \begin{cases}
        -u''=f\quad&\text{in }\Omega=(-1,1),\\
        u(-1)=u_0(-1),\quad&\\
        u'(1)=g(1),\quad&\\
    \end{cases}
$$
with $f(x)=9\sin(3x+3)$, $u_0(-1)=0$, and $g(1)=3\cos(6)$. The exact solution is $u(x)=\sin(3x+3)$.

In [10]:
import numpy as np
from matplotlib import pyplot as plt
import sympy as sp

## (a)

Derive a weak (Galerkin) formulation for the problem.

* $V=\{v:\Omega \rightarrow \real, v \in \mathcal{H}^1, v(-1)=0 \}$
* $a(u,v)=\int_{\Omega}u^{\prime}(x)v^{\prime}(x)dx$
* $b(v)=\int_{\Omega}f(x)v(x)dx + 3*cos(6)*v(1)$

# Well-posedness ? 

We have to prove :

1. the problem has a solution
2. this solution is unique
3. The solution depends continuously on the data and the parameters



## (b)

Implement a function that computes the four mesh-related quantities `grid`, `elements`, `endpoints`, and `boundaries`.

In [11]:
def createmesh1D(h, domain=(-1, 1)):

    a, b = domain
    N = int((b - a) / h)
    h = (b - a) / N  # Adjust h to ensure integer number of elements
    
    grid = np.linspace(a, b, N + 1)
    elements = [[i, i + 1] for i in range(N)]
    endpoints = [0, N]
    boundaries = [a, b]
    boundaries_bs = [1,2,3]
    return grid, elements, endpoints, boundaries, boundaries_bs

In [12]:
# check your implementation of createmesh1D
from pickle import load
data = createmesh1D(0.1)
with open("lab1/check/createmesh1D_k1.pkl", "rb") as f: data_ex = load(f)
for dat, dat_ex, name in zip(data, data_ex,
                             ["grid", "elements", "endpoints", "boundaries"]):
    print(f"'{name}' should be {dat_ex}\ncomputed '{name}' is {dat}\n")

'grid' should be [-1.  -0.9 -0.8 -0.7 -0.6 -0.5 -0.4 -0.3 -0.2 -0.1  0.   0.1  0.2  0.3
  0.4  0.5  0.6  0.7  0.8  0.9  1. ]
computed 'grid' is [-1.  -0.9 -0.8 -0.7 -0.6 -0.5 -0.4 -0.3 -0.2 -0.1  0.   0.1  0.2  0.3
  0.4  0.5  0.6  0.7  0.8  0.9  1. ]

'elements' should be [[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8], [8, 9], [9, 10], [10, 11], [11, 12], [12, 13], [13, 14], [14, 15], [15, 16], [16, 17], [17, 18], [18, 19], [19, 20]]
computed 'elements' is [[0, 1], [1, 2], [2, 3], [3, 4], [4, 5], [5, 6], [6, 7], [7, 8], [8, 9], [9, 10], [10, 11], [11, 12], [12, 13], [13, 14], [14, 15], [15, 16], [16, 17], [17, 18], [18, 19], [19, 20]]

'endpoints' should be [0, 20]
computed 'endpoints' is [0, 20]

'boundaries' should be [1, 1]
computed 'boundaries' is [-1, 1]



## (c)

Implement a function that assembles the P1-FEM LHS matrix.

In [13]:
def assemble_LHS_poisson(mesh):
    
    grid, elements, endpoints, boundaries, boundaries_bs = mesh

    K_ref = np.array([[-1., 1.], [1., -1.]])
    N=len(grid)
    K = np.zeros((N,N))

    for el in elements :

        i, j = el
        h = grid[i] - grid[j]
        K_temp =  (1. / h) * K_ref

        K[i,i] += K_temp[0,0]
        K[i,j] += K_temp[0,1]
        K[j,i] += K_temp[1,0]
        K[j,j] += K_temp[1,1]
    for j, flag in zip(endpoints, boundaries_bs):
        if flag == 1:  # Dirichlet
            K[j, :] = 0
            K[j, j] = 1
        elif flag == 3:  # Robin
            K[j, j] += 1  # Ajout du terme Robin (à modifier selon la condition exacte)
    return K

In [14]:
# check your implementation of assemble_LHS_poisson
from pickle import load
mat = assemble_LHS_poisson(createmesh1D(0.1))
with open("lab1/check/assemble_LHS_poisson.pkl", "rb") as f: mat_ex = load(f)
print(f"stiffness matrix should be {mat_ex}\ncomputed stiffness matrix is {mat}")

stiffness matrix should be [[ 1.  0.  0.  0.  0.]
 [-2.  4. -2.  0.  0.]
 [ 0. -2.  4. -2.  0.]
 [ 0.  0. -2.  4. -2.]
 [ 0.  0.  0. -2.  2.]]
computed stiffness matrix is [[  1.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.]
 [-10.  20. -10.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.]
 [  0. -10.  20. -10.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.]
 [  0.   0. -10.  20. -10.   0.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0. -10.  20. -10.   0.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0. -10.  20. -10.   0.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0. -10.  20. -10.   0.   0.   0.   0.   0.   0.
    0.   0.   0.   0.   0.   0.   0.]
 [  0.   0.   0.   0.   0.   0. -10.  20. -10.   0.   0.  

## (d)

Write a function that assembles the P1-FEM RHS vector.

In [15]:
"""def quadrature1D(domain, integrand):

    Function to perform quadrature of callable function integrand over interval specified by domain.
        \int_{domain} integrand(x) dx
    """

def quadrature1D(domain, integrand):

    a, b = domain

    # Quadrature de Gauss-Legendre, ordre 2
    nodes = np.array([-1/np.sqrt(3), 1/np.sqrt(3)])  # Points de Gauss-Legendre
    weights = np.array([1, 1])  # Poids associés
    transformed_nodes = 0.5 * (b - a) * nodes + 0.5 * (a + b)  # Mapping vers [a, b]
    integral = sum(weights[i] * integrand(transformed_nodes[i]) for i in range(2))
    return 0.5 * (b - a) * integral  # Facteur de changement de variable
    
def assemble_RHS_poisson(mesh, u0, f, g):
    """
    Function to assemble P1-FEM RHS.
    """
    grid, elements, endpoints, boundaries = mesh
    ...
    vec = np.zeros(len(grid))
    for el in elements: # volume terms (must integrate)
        ...
    for j, b in zip(endpoints, boundaries): # all BCs
        # j is index of DoF, b is boundary b
        ...
    return vec

<>:1: SyntaxWarning: invalid escape sequence '\i'
<>:1: SyntaxWarning: invalid escape sequence '\i'
/tmp/ipykernel_22100/685703123.py:1: SyntaxWarning: invalid escape sequence '\i'
  """def quadrature1D(domain, integrand):


In [16]:
# check your implementation of quadrature1D
from pickle import load
val = quadrature1D([0., 1.], lambda x: 4 * x ** 3)
print(f"integral value is 1.\nuser-approximated value is {val}")
print("NOTE: results may vary slightly depending on the implemented quadrature formula")

integral value is 1.
user-approximated value is 0.9999999999999999
NOTE: results may vary slightly depending on the implemented quadrature formula


In [17]:
# check your implementation of assemble_RHS_poisson
from pickle import load
vec = assemble_RHS_poisson(createmesh1D(0.1), lambda x: 0.,
                           lambda x: 9 * np.sin(3 * x + 3),
                           lambda x: 3 * np.cos(6))
with open("lab1/check/assemble_RHS_poisson.pkl", "rb") as f: vec_ex = load(f)
print(f"RHS vector should be {vec_ex}\ncomputed RHS vector is {vec}")
print("NOTE: results may vary slightly depending on the implemented quadrature formula")

/tmp/ipykernel_22100/685703123.py:1: SyntaxWarning: invalid escape sequence '\i'
  """def quadrature1D(domain, integrand):


ValueError: too many values to unpack (expected 4)

## (e)

Compute and plot the P1-FE solution.

In [ ]:
def solve_poisson(h):
    """
    Function to compute the P1-FEM solution uh with mesh-size h.
    """
    ...
...

Compute the FE error in the $L^\infty(\Omega)$ norm

In [ ]:
def eval_uh(mesh, uh, x):
    """
    Function to evaluate the P1-FEM solution uh (based on mesh) at new point(s) x.
    """
    ...
...

Compute the FE error in the $L^2(\Omega)$- and $H^1(\Omega)$-norms.

In [ ]:
def eval_uh_prime(mesh, uh, x):
    """
    Function to evaluate the first derivative of the P1-FEM solution uh (based on mesh) at new point(s) x.
    """
    ...
...

## (f)

Solve the problem on a sequence of finer and finer meshes. Identify the convergence rates of the errors.

In [ ]:
...
hs = .5 ** np.arange(7)
for h in hs:
    ...
...

## (g)

Solve the problem on a sequence of finer and finer meshes. Identify the convergence rates of the FEM matrix's condition number.

In [ ]:
...
hs = .5 ** np.arange(7)
for h in hs:
    ...
...

## (h)

Derive two weak formulations for a modified problem with non-homogeneous Dirichlet BCs.

1) Petrov-Galerkin
    ...

2) Lifting.
    ...

Solve the non-homogeneous problem and study the convergence rates.

In [ ]:
def solve_poisson_nonhomogeneous(h):
    """
    Function to compute the P1-FEM solution uh of the problem with non-homogeneous Dirichlet BCs with mesh-size h.
    """
    ...
...
hs = .5 ** np.arange(7)
for h in hs:
    ...
...

## (i)

Derive a weak formulation for a modified problem with a Robin BC. Study the convergence rates.

* $a(u,v)=...$

In [ ]:
def solve_poisson_robin(h):
    """
    Function to compute the P1-FEM solution uh of the problem with Robin BCs with mesh-size h.
    """
    ...
...
hs = .5 ** np.arange(7)
for h in hs:
    ...
...